# MLflow: tracking

In [ ]:
import pandas as pd
import sklearn.pipeline as pipeline
import sklearn.compose as compose
import sklearn.preprocessing as preprocessing
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection 
import sklearn.metrics as metrics
import mlflow
import mlflow.tracking as tracking
import mlflow.sklearn
import numpy as np

In [ ]:
MLFLOW_EXPERIMENT = "Tracking demo"

## Read in data

In [ ]:
df = pd.read_csv("../data/processed/train.csv")

Create features and target.

In [ ]:
X = df.drop("Attrition", axis=1)
y = df.loc[:, "Attrition"]

## Preprocessing pipeline

In [ ]:
categorical_cols = []
numerical_cols = []

for colname, coltype in X.dtypes.items():
    if coltype == "object":
        categorical_cols.append(colname)
    # In our data there are only 2 types
    else:
        numerical_cols.append(colname)
    
print(f"Categorical: {', '.join(categorical_cols)}")
print(f"Numerical: {', '.join(numerical_cols)}")

In [ ]:
scaler = preprocessing.StandardScaler()
oh_encoder = preprocessing.OneHotEncoder(categories="auto", drop="first")

preprocess_pipeline = compose.ColumnTransformer(transformers=[ # (name, transformer, column(s))
    ("scaler", scaler, numerical_cols),
    ("one_hot_encode", oh_encoder, categorical_cols)
], remainder="drop")

Fit the pipeline.

In [ ]:
X = preprocess_pipeline.fit_transform(X)

Get columns names.

In [ ]:
oh_encoded_categorical_colnames = preprocess_pipeline.named_transformers_["one_hot_encode"].get_feature_names().tolist()

transformed_colnames = numerical_cols + oh_encoded_categorical_colnames

Make target binary.

In [ ]:
y = y.apply(lambda y: 1 if y == "Yes" else 0)

## Train a model

### Set up MLflow experiment

Check if an experiment already exists.

In [ ]:
client = tracking.MlflowClient()
experiments = client.list_experiments()
if MLFLOW_EXPERIMENT in [e.name for e in experiments]:
    print("Experiment already exists.")
else:
    print("Experiment does not exist. Creating it.")
    mlflow.create_experiment(MLFLOW_EXPERIMENT)

Set experiment.

In [ ]:
mlflow.set_experiment(MLFLOW_EXPERIMENT)

### Run MLflow experiment

Define a grid of hyperparameters.

In [ ]:
for alpha in [0.001, 0.01, 0.1, 1.0, 10.0]:
    for l1_ratio in np.arange(0.1, 1, 0.1):
        with mlflow.start_run():
            # Define model
            model = linear_model.ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
            
            # Compute CV scores
            scores = model_selection.cross_validate(
                model, 
                X, 
                y, 
                cv=5, 
                scoring={
                    "roc_auc": metrics.make_scorer(metrics.roc_auc_score),
                    "log_loss": metrics.make_scorer(metrics.log_loss)
                }
            )
            
            # Log paramas
            mlflow.log_params({
                "alpha": alpha,
                "l1_ratio": l1_ratio
            })            
            
            # Log metrics
            mlflow.log_metrics({
                "val_roc_auc": scores["test_roc_auc"].mean(),
                "val_log_loss": scores["test_log_loss"].mean(),
            })
            
            # Log model tags, e.g., model type
            mlflow.set_tags({
                "model": "ElasticNet"
            })
            
            # Log model itself
            mlflow.sklearn.log_model(model, artifact_path="model")
            
            # Log preprocessing pipeline
            mlflow.sklearn.log_model(preprocess_pipeline, artifact_path="preprocess_pipeline")